# Evaluation using RAGAS

###Evaluating using a Non-LLM Metric

In [1]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter."
}

metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

d:\My Files\Complete_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.13718598426177148

###LLM

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [4]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_community.embeddings import OllamaEmbeddings

evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embedding = LangchainEmbeddingsWrapper(OllamaEmbeddings(model="llama3.2:1b"))

C:\Users\user\AppData\Local\Temp\ipykernel_7580\1617431089.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  evaluator_embedding = LangchainEmbeddingsWrapper(OllamaEmbeddings(model="llama3.2:1b"))


###Evaluation

In [5]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(name="summary_accuracy",llm=evaluator_llm, definition="Verify if the summary is accurate.")
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

1

###Using Dataset with examples

In [6]:
from datasets import load_dataset
from ragas import EvaluationDataset
eval_dataset = load_dataset("explodinggradients/earning_report_summary",split="train")
eval_dataset = EvaluationDataset.from_hf_dataset(eval_dataset)
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))

d:\My Files\Complete_RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--explodinggradients--earning_report_summary. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 50/50 [00:00<00:00, 879.80 examples/s]

Features in dataset: ['user_input', 'response']
Total samples in dataset: 50


In [7]:
from ragas.evaluation import evaluate

results = evaluate(eval_dataset, metrics=[metric])
results

Evaluating: 100%|██████████| 50/50 [01:20<00:00,  1.61s/it]


{'summary_accuracy': 1.0000}

In [8]:
results.to_pandas()

,user_input,response,summary_accuracy
0,summarise given text\nThe Q2 earnings report r...,The Q2 earnings report showed a 15% revenue in...,1
1,"summarise given text\nIn 2023, North American ...",Companies are strategizing to adapt to market ...,1
2,"summarise given text\nIn 2022, European expans...",Many companies experienced a notable 15% growt...,1
3,summarise given text\nSupply chain challenges ...,"Supply chain challenges in North America, caus...",1
4,"summarise given text\nIn Q2 2023, the company ...",The company experienced a notable increase in ...,1
5,"summarise given text\nIn 2023, marketing campa...","In 2023, marketing campaigns in North America ...",1
6,summarise given text\nThe company's internatio...,The company's international expansion strategy...,1
7,"summarise given text\nIn 2024, companies are i...",Companies are using data analytics to customiz...,1
8,"summarise given text\nIn 2023, logistics inves...",Driven by technological and infrastructural ad...,1
9,"summarise given text\nIn 2023, the company exp...",The company faced challenges due to competitio...,1
